# King County Housing Data – Exploratory Data Analysis (EDA)

This notebook explores the King County Housing dataset.  
The goal is to understand the data, find patterns, validate hypotheses, and provide insights and recommendations for a client.

We will follow the EDA checklist:
ich wdjojwd oid  id id 

1. Understanding  
2. Hypothesis  
3. Explore  
4. Clean  
5. Relationships  
6. Back to the Hypothesis  
7. Fine Tune  
8. Explain
9. Information & links

In [13]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Display settings (optional, simple)
pd.set_option('display.max_columns', None)
sns.set(style='whitegrid')


In [14]:
# Replace filenames if needed
sales = pd.read_csv("data/king_county_house_sales_202512091732.csv")
details = pd.read_csv("data/king_county_house_details_202512091731.csv")

sales.head(), details.head()


(         date     price    house_id  id
 0  2014-10-13  221900.0  7129300520   1
 1  2014-12-09  538000.0  6414100192   2
 2  2015-02-25  180000.0  5631500400   3
 3  2014-12-09  604000.0  2487200875   4
 4  2015-02-18  510000.0  1954400510   5,
            id  bedrooms  bathrooms  sqft_living  sqft_lot  floors  waterfront  \
 0     1000102       6.0        3.0       2400.0    9373.0     2.0         NaN   
 1   100100050       3.0        1.0       1320.0   11090.0     1.0         0.0   
 2  1001200035       3.0        1.0       1350.0    7973.0     1.5         NaN   
 3  1001200050       4.0        1.5       1260.0    7248.0     1.5         NaN   
 4  1003000175       3.0        1.0        980.0    7606.0     1.0         0.0   
 
    view  condition  grade  sqft_above  sqft_basement  yr_built  yr_renovated  \
 0   0.0          3      7      2400.0            0.0      1991           0.0   
 1   0.0          3      7      1320.0            0.0      1955           0.0   
 2   0.0        

## 1. Understanding the Data

We check the structure, column names, and data types.


In [17]:
sales.info()
details.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21597 entries, 0 to 21596
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   date      21597 non-null  object 
 1   price     21597 non-null  float64
 2   house_id  21597 non-null  int64  
 3   id        21597 non-null  int64  
dtypes: float64(1), int64(2), object(1)
memory usage: 675.0+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21420 entries, 0 to 21419
Data columns (total 19 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             21420 non-null  int64  
 1   bedrooms       21420 non-null  float64
 2   bathrooms      21420 non-null  float64
 3   sqft_living    21420 non-null  float64
 4   sqft_lot       21420 non-null  float64
 5   floors         21420 non-null  float64
 6   waterfront     19060 non-null  float64
 7   view           21357 non-null  float64
 8   condition      21420 non-null  int64  
 

01.  checking the merged data we will Do light cleaning first — NOT full cleaning — then explore and refine.
02. but as first I prepare df2 to compare how it looks if we use as unique Id the house id and not the sales id 

In [19]:
# Merge alternative mit Suffixen
df2 = sales.merge(
    details, 
    left_on="house_id", 
    right_on="id", 
    how="left",
    suffixes=('_sale', '_detail')
)

# Umbenennen für Klarheit
df2.rename(columns={
    "id_detail": "house_id",  # eindeutige Haus-ID
    "id_sale": "sales_id",    # Transaktions-ID
}, inplace=True)

# Überprüfung
print(df2.shape)
print(df2.columns)
df2.head()

(21597, 23)
Index(['date', 'price', 'house_id', 'sales_id', 'house_id', 'bedrooms',
       'bathrooms', 'sqft_living', 'sqft_lot', 'floors', 'waterfront', 'view',
       'condition', 'grade', 'sqft_above', 'sqft_basement', 'yr_built',
       'yr_renovated', 'zipcode', 'lat', 'long', 'sqft_living15',
       'sqft_lot15'],
      dtype='object')


,date,price,house_id,sales_id,house_id,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,2014-10-13,221900.0,7129300520,1,7129300520,3.0,1.00,1180.0,5650.0,1.0,NaN,0.0,3,7,1180.0,0.0,1955,0.0,98178,47.5112,-122.257,1340.0,5650.0
1,2014-12-09,538000.0,6414100192,2,6414100192,3.0,2.25,2570.0,7242.0,2.0,0.0,0.0,3,7,2170.0,400.0,1951,19910.0,98125,47.7210,-122.319,1690.0,7639.0
2,2015-02-25,180000.0,5631500400,3,5631500400,2.0,1.00,770.0,10000.0,1.0,0.0,0.0,3,6,770.0,0.0,1933,NaN,98028,47.7379,-122.233,2720.0,8062.0
3,2014-12-09,604000.0,2487200875,4,2487200875,4.0,3.00,1960.0,5000.0,1.0,0.0,0.0,5,7,1050.0,910.0,1965,0.0,98136,47.5208,-122.393,1360.0,5000.0
4,2015-02-18,510000.0,1954400510,5,1954400510,3.0,2.00,1680.0,8080.0,1.0,0.0,0.0,3,8,1680.0,0.0,1987,0.0,98074,47.6168,-122.045,1800.0,7503.0


## 2. Hypotheses

Before exploring the data, here are some assumptions:

1. Houses closer to the water are more expensive.
2. Houses with more bedrooms and bathrooms have higher prices.
3. Some ZIP codes might form “rich neighborhoods”.
4. Newer, renovated houses should have higher prices.

These will be tested during EDA.


# use miro bort to collect all information 
https://miro.com/app/board/uXjVGd0Xl8o=/

## 3. Explore the Data

We look for:
- Missing values  
- Outliers  
- Strange values  
- Distribution of key columns  


In [ ]:
sales.isna().sum(), details.isna().sum()


In [ ]:
sales.describe()


In [ ]:
df = sales.merge(details, on="id", how="left")
df.head()


In [ ]:
sns.histplot(df["price"], bins=50)
plt.title("Price Distribution")
plt.xlabel("Price")
plt.ylabel("Count")
plt.show()


## 4. Cleaning the Data

We check:
- Incorrect values  
- Missing values  
- Outliers  
- Whether we need to transform variables (log, categories, etc.)


In [ ]:
sns.boxplot(x=df["price"])
plt.title("Price Outliers")
plt.xlabel("Price")
plt.show()


## 5. Relationships

We explore correlations between variables.


In [ ]:
plt.figure(figsize=(12,8))
# Select numeric columns to avoid errors
numeric_cols = df.select_dtypes(include=[np.number])
sns.heatmap(numeric_cols.corr(), annot=False, cmap="coolwarm")
plt.title("Correlation Heatmap (Numeric Columns)")
plt.show()


## 6. Back to the Hypotheses

We check if our assumptions are true or not and update them if necessary.


In [ ]:
sns.scatterplot(data=df, x="sqft_living", y="price")
plt.title("Living Area vs Price")
plt.xlabel("Square Feet Living")
plt.ylabel("Price")
plt.show()


## 7. Fine Tune

Remove unnecessary plots, make visuals clear, add labels, and prepare clean results for the client.


## 8. Explain – Insights & Recommendations

### Insights (at least 3)
- Insight 1  
- Insight 2  
- Insight 3  

### Geographic Insight
- ZIP code or location-based finding  

### Recommendations (at least 3)
- Rec 1  
- Rec 2  
- Rec 3  

Client chosen: **(your choice)**
